### 처음부터 훈련 루프 작성

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

#### 소개

Keras는 기본 학습 및 평가 루프 인 fit() 및 evaluate() 합니다. 

이제 훈련 및 평가에 대한 매우 낮은 수준의 제어를 원한다면 처음부터 자신의 훈련 및 평가 루프를 작성해야합니다. 이것이이 가이드의 내용입니다.

### GradientTape 사용 : 첫 번째 종단 간 예제

GradientTape 범위 내에서 모델을 호출하면 손실 값과 관련하여 학습 가능한 레이어 가중치의 기울기를 검색 할 수 있습니다. 옵티 마이저 인스턴스를 사용하면 이러한 기울기를 사용하여 이러한 변수를 업데이트 할 수 있습니다 ( model.trainable_weights 사용하여 검색 할 수 model.trainable_weights ).

간단한 MNIST 모델을 고려해 보겠습니다.

In [3]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

맞춤형 훈련 루프와 함께 미니 배치 그래디언트를 사용하여 훈련 해 봅시다.

먼저 옵티 마이저, 손실 함수 및 데이터 세트가 필요합니다.

In [4]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

다음은 훈련 루프입니다.

- 에포크를 반복하는 for 루프를 엽니다.
- 각 Epoch에 대해 데이터 세트를 일괄 적으로 반복하는 for 루프를 엽니다.
- 각 배치에 대해 GradientTape() 범위를 엽니다.
- 이 범위 내에서 모델 (순방향 전달)을 호출하고 손실을 계산합니다.
- 범위 밖에서 손실과 관련하여 모델 가중치의 기울기를 검색합니다.
- 마지막으로 최적화 도구를 사용하여 기울기를 기반으로 모델의 가중치를 업데이트합니다.

In [6]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        with tf.GradientTape() as tape:

            logits = model(x_batch_train, training=True)  

            loss_value = loss_fn(y_batch_train, logits)

        grads = tape.gradient(loss_value, model.trainable_weights)

        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))


Start of epoch 0
Training loss (for one batch) at step 0: 0.2779
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.3419
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.4533
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.1323
Seen so far: 38464 samples

Start of epoch 1
Training loss (for one batch) at step 0: 0.3922
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.2796
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.3279
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.4134
Seen so far: 38464 samples


### 낮은 수준의 메트릭 처리

이 기본 루프에 메트릭 모니터링을 추가해 보겠습니다.

처음부터 작성된 교육 루프에서 기본 제공 메트릭 (또는 사용자가 작성한 사용자 지정 메트릭)을 쉽게 재사용 할 수 있습니다. 흐름은 다음과 같습니다.

- 루프 시작시 메트릭 인스턴스화
- 각 배치 후 metric.update_state() 호출
- 메트릭의 현재 값을 표시해야하는 경우 metric.result() 호출하십시오.
- 메트릭의 상태를 지워야 할 때 (일반적으로 epoch의 끝에서 metric.reset_states() 호출 metric.reset_states()

이 지식을 사용하여 각 SparseCategoricalAccuracy 가 끝날 때 유효성 검사 데이터에 대한 SparseCategoricalAccuracy 를 계산해 SparseCategoricalAccuracy .

In [7]:
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

optimizer = keras.optimizers.SGD(learning_rate=1e-3)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

다음은 훈련 및 평가 루프입니다.

In [8]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        train_acc_metric.update_state(y_batch_train, logits)

        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 89.8828
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.8284
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.6240
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.5073
Seen so far: 38464 samples
Training acc over epoch: 0.7777
Validation acc: 0.8416
Time taken: 2.42s

Start of epoch 1
Training loss (for one batch) at step 0: 1.2146
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0182
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.6555
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.7076
Seen so far: 38464 samples
Training acc over epoch: 0.8688
Validation acc: 0.8819
Time taken: 2.42s


## tf.function 훈련 단계 tf.function

TensorFlow 2.0의 기본 런타임은 즉시 실행 입니다. 따라서 위의 훈련 루프는 열심히 실행됩니다.

이것은 디버깅에 적합하지만 그래프 컴파일에는 확실한 성능 이점이 있습니다. 계산을 정적 그래프로 설명하면 프레임 워크가 전역 성능 최적화를 적용 할 수 있습니다. 프레임 워크가 다음에 무슨 일이 일어날 지 알지 못한 채 탐욕스럽게 한 작업을 차례로 실행하도록 제한되어 있으면 불가능합니다.

텐서를 입력으로 사용하는 모든 함수를 정적 그래프로 컴파일 할 수 있습니다. 다음과 같이 @tf.function 데코레이터를 추가하면됩니다.

In [9]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

평가 단계에서도 똑같이합시다.

In [11]:
@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

이제이 컴파일 된 학습 단계를 사용하여 학습 루프를 다시 실행 해 보겠습니다.

In [12]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 0.2865
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.4082
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.3934
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.3977
Seen so far: 38464 samples
Training acc over epoch: 0.8952
Validation acc: 0.8775
Time taken: 0.67s

Start of epoch 1
Training loss (for one batch) at step 0: 0.2144
Seen so far: 64 samples
Training loss (for one batch) at step 200: 0.1573
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.4753
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 0.1794
Seen so far: 38464 samples
Training acc over epoch: 0.9098
Validation acc: 0.9112
Time taken: 0.47s


훨씬 빠르죠?